In [167]:
import matrix as m
import copy 

#utils
def getDirectionOfMovement(start_point, end_point):
    """
    Moves the character from the start_point to the end_point and returns the direction of the move
    (L, R, U, D)
    """
    direction = ""
    if start_point[0] == end_point[0]:
        if start_point[1] > end_point[1]:
            direction = "L"
        else:
            direction = "R"
    else:
        if start_point[0] > end_point[0]:
            direction = "U"
        else:
            direction = "D"

    return direction

#given an array delete the element if the next is the same as the current
def deleteDuplicates(array):
    i = 0
    while i < len(array) - 1:
        if array[i] == array[i + 1]:
            del array[i]
            i -= 1
        i += 1
    return array


class BlockInterface:
    def __init__(self, is_collectable, is_movable, is_walkable, name, gameBoard, is_fillable = False):
        #document the block
        """
        Parameters
        ----------
        is_collectable : bool
            Whether the block can be collected by the player for example keys or diamonds.
        is_movable : bool
            Whether the block can be moved by the player or not.
        is_walkable : bool
            Whether the player can walk on the block or not.
        """
        
        self._is_collectable = is_collectable
        self._is_movable = is_movable
        self._is_walkable = is_walkable 
        self._is_fillable = is_fillable
        self._name = name
        self._gameBoard = gameBoard
        self._rock = None #atribute to save a rock if the block is a rock
        self._location = None #atribute to save the location of the block
        self.interacted_step = None

    def __repr__(self) -> str:
        return str(self.name) + " at " + str(self.get_position())  

    def get_interacted_step(self):
        return self.interacted_step

    def set_interacted_step(self, step):
        self.interacted_step = step


    def has_a_rock(self):
        return self._rock != None #objects that has a rock cannot be reachable or walkable
        
    def has_close_rocks(self):
        location = self.get_position()
        closer_rocks = self.get_gameBoard().getCloserRocks(location)
        if len(closer_rocks) > 0:
            return True
        return False

    def get_close_rocks(self):
        location = self.get_position()
        return self.get_gameBoard().getCloserRocks(location)
    
    def get_rock(self):
        return self._rock

    def can_contain_rock(self):
        return self._is_fillable

    def set_rock(self, rock):
        self._rock = rock

    def get_position(self):
        return self._location

    def set_location(self, location):
        self._location = location

    def walkOver(self):
        pass

    def unwalkOver(self):
        pass

    def interact(self, step = None):
        pass

    def unInteract(self, step = None):
        pass

    def isReachable(self):
        return False


    def set_is_collectable(self, is_collectable):
        self._is_collectable = is_collectable
    
    def get_is_collectable(self):
        return self._is_collectable

    is_collectable = property(get_is_collectable, set_is_collectable)
    
    def set_is_movable(self, is_movable):
        self._is_movable = is_movable

    def get_is_movable(self):
        return self._is_movable

    is_movable = property(get_is_movable, set_is_movable)

    def set_is_walkable(self, is_walkable):
        self._is_walkable = is_walkable

    def get_is_walkable(self):
        return self._is_walkable and not self.has_a_rock()

    is_walkable = property(get_is_walkable, set_is_walkable)

    def get_gameBoard(self):
        return self._gameBoard

    def set_gameBoard(self, gameBoard):
        self._gameBoard = gameBoard
    
    gameBoard = property(get_gameBoard, set_gameBoard)

    def get_name(self):
        return self._name

    def set_name(self, name):
        self._name = name

    name = property(get_name, set_name)
    
    def __str__(self):
        return self.name[0:1]

class Wall(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Wall", gameBoard)

class Floor(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Floor", gameBoard)

    def isReachable(self):
        return super().has_close_rocks() == True
        

class Destination(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, False, "Destination", gameBoard)

    def isReachable(self):
        #if there is no more diamonds to collect, the destination is reachable
        return super().get_gameBoard().remainingDiamonds == 0

class Key(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, True, "Key", gameBoard)

    def interact(self, step):
        if super().get_gameBoard().has_key == True:
            return
        if super().get_interacted_step() != None:
            return
        super().set_interacted_step(step)
        super().get_gameBoard().has_key = True
        super().set_is_collectable(False)

    def unInteract(self, step):
        if super().get_gameBoard().has_key == False:
            return
        if super().get_interacted_step() != step:
            return
        super().set_interacted_step(None)
        super().get_gameBoard().has_key = False
        super().set_is_collectable(True)

    def isReachable(self):
        if super().has_a_rock():
            return False
        if super().has_close_rocks():
            return True
        return super().get_is_collectable() and super().get_gameBoard().has_key == False


class Diamond(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, True, "Diamond", gameBoard)
    
    def interact(self, step):
        if self.has_a_rock():
            return
        if super().get_interacted_step() != None:
            return
        super().set_interacted_step(step)
        super().get_gameBoard().remainingDiamonds -= 1
        super().set_is_collectable(False)
        super().get_gameBoard().updateDestinationState()

    def unInteract(self, step):
        if super().get_interacted_step() != step:
            return
        super().set_interacted_step(None)
        super().get_gameBoard().remainingDiamonds += 1
        super().set_is_collectable(True)
        super().get_gameBoard().updateDestinationState()
    
    def isReachable(self):
        if super().has_a_rock():
            return False
        if super().has_close_rocks():
            return True
        return super().get_is_collectable()


class KeyDoor(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, False, "KeyDoor", gameBoard)
        self.is_open = False
    
    def isReachable(self):
        return super().get_gameBoard().has_key == True and super().get_is_collectable() == True

    def interact(self, step):
        if super().get_gameBoard().has_key == False:
            return
        if super().get_interacted_step() != None:
            return
        super().set_interacted_step(step)
        super().get_gameBoard().has_key = False
        super().set_is_collectable(False)
        super().set_is_walkable(True)

    def unInteract(self, step):
        if super().get_interacted_step() != step:
            return
        super().set_interacted_step(None)
        super().get_gameBoard().has_key = True
        super().set_is_collectable(True)
        super().set_is_walkable(False)

    def get_is_walkable(self):
        return super().get_gameBoard().has_key

    def set_is_walkable(self, is_walkable):
        super().set_is_walkable(is_walkable)

    is_walkable = property(get_is_walkable, set_is_walkable)

class Spikes(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Spikes", gameBoard)

    def walkOver(self):
        super().set_is_walkable(False)

    def unwalkOver(self):
        super().set_is_walkable(True)

    def isReachable(self):
        if super().has_close_rocks():
            return True

class Rock(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, True, False, "Rock", gameBoard)
        self.visited = super().get_gameBoard().generateVisitedMatrix()

    def move(self, from_block, to_block):
        #if to block is a child from Rock object, walkOver is called
        if isinstance(to_block, Rock):
            to_block.walkOver()
    
    def visit(self, position):
        row, col = position
        self.visited[row][col] = True

    def unVisit(self, position):
        row, col = position
        self.visited[row][col] = False

    def wasVisited(self, point):
        row, col = point
        return self.visited[row][col]

    def walkOver(self):
        pass

    def isReachable(self):
        return True

class Magma(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Magma", gameBoard)

    def walkOver(self):
        pass

class Hole(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Hole", gameBoard, True)

    def get_is_walkable(self):
        return super().has_a_rock()

    def set_is_walkable(self, is_walkable):
        super().set_is_walkable(is_walkable)

    def isReachable(self):
        if super().has_close_rocks() and self.get_is_walkable():
            return True
        return False

    is_walkable = property(get_is_walkable, set_is_walkable)
        

class Button(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Button", gameBoard)

    def walkOver(self):
        pass

class DoorButton(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "DoorButton", gameBoard)

    def walkOver(self):
        pass

class Character(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Character", gameBoard)

    def walkOver(self):
        pass



class BlockGenerator:
    """
    Class to generate all the objects based on a fixed map
    """
    def __init__(self, gameBoard):
        self.map = {
            'R' : self._create_rock,
            'D' : Diamond,
            'P' : Floor,
            'M' : Wall,
            'W' : Magma,
            'A' : Destination,
            'C' : Floor,
            'K' : KeyDoor,
            'L' : Key,
            'S' : Spikes,
            'H' : Hole,
            'B' : Button,
            'U' : DoorButton,
        }
        self.gameBoard = gameBoard

    def _create_rock(self, gameBoard):
        rock = Rock(gameBoard)
        floor = Floor(gameBoard)
        floor.set_rock(rock)
        return floor

    def generate(self, target):
        """
        returns a new Object based on the given char
        for example:
        'R' -> Rock
        'D' -> Diamond
        """
        return  self.map[target](self.gameBoard)

class ReachableBlock():
    """
    Class to store the reachable blocks
    
    Parameters:
    ----------
    block_from: BlockInterface
        the block to where starts the path
    block_to: BlockInterface
        the block to where ends the path
    path: list
        the path to go from block_from to block_to
    """
    def __init__(self, block_from, block_to, path):
        self._block_from = block_from
        self._block_to = block_to
        self._path = path

    def _get_distance(self):
        return len(self._path)
    
    def _get_block_from(self):
        return self._block_from

    def _get_block_to(self):
        return self._block_to
    
    def _get_path(self):
        return self._path

    def _get_list_movements(self):
        list_movements = []
        for i in range(len(self.path) - 1):
            start_point = self.path[i]
            end_point = self.path[i+1]
            moves = getDirectionOfMovement(start_point, end_point)
            list_movements.append(moves)

        return list_movements

    distance = property(_get_distance)
    block_from = property(_get_block_from)
    block_to = property(_get_block_to)
    path = property(_get_path)
    list_movements = property(_get_list_movements)

    def __repr__(self) -> str:
        return f"ReachableBlock({self.block_from}, {self.block_to}, {self.path})"

class GameBoard:
    def __init__(self, gameMap):
        self.has_key = False
        self.remainingDiamonds = 0
        self.characterLocation = (0,0)
        self.destinationLocation = (0,0)
        self.board = self.generateBoard(gameMap, self)

    def getBlock(self, point) -> BlockInterface:
        """
        Parameters
        ----------
        point : Tuple(Int)
            The point to return in the map
        """ 
        return self.board[point[0]][point[1]]

    def getShortestPathsFromRocks(self, rock_location, player_location):
        """
        Parameters
        ----------
        point : Tuple(Int)
            The point to return in the map
        """ 
        visited = self.generateVisitedMatrix()
        visited[rock_location[0]][rock_location[1]] = True
        player_path = []
        shortest_paths = {}
        self.getShortestPathsFromRocks_recursive(rock_location, player_location, visited, player_path, shortest_paths)
        #iterate over each element of shortest path and if the previus position is the same as the current position, remove it
        for key in shortest_paths:
            path = shortest_paths[key]
            deleteDuplicates(path)
        return shortest_paths


    def getShortestPathsFromRocks_recursive(self, rock_location, player_location, visited, player_path, shortest_paths):
        """
        Parameters
        ----------
        point : Tuple(Int)
            The point to return in the map
        """ 
        if self.getBlock(rock_location).can_contain_rock():
            if rock_location not in shortest_paths:
                shortest_paths[rock_location] = copy.deepcopy(player_path)
            else:
                if len(player_path) < len(shortest_paths[rock_location]):
                    shortest_paths[rock_location] = copy.deepcopy(player_path)
            return 

        #generate the moveable points of the rock
        movable_points = self.getMovablePointRock(rock_location)
        #get the shortest path to each point
        shortest_player_paths = self.getShortestPathFromTo(player_location, movable_points)
        for close_to_rock_point in shortest_player_paths:
            #move the rock to the new point
            new_rock_location_block = self.moverRock(self.getBlock(close_to_rock_point), self.getBlock(rock_location))
            new_rock_location = new_rock_location_block.get_position()
            #if was alredy visited, move back
            if visited[new_rock_location[0]][new_rock_location[1]]:
                self.moveBackRock(self.getBlock(close_to_rock_point), self.getBlock(rock_location))
                continue
            #path that use the player to get to the rock
            path_close_rock_point = shortest_player_paths[close_to_rock_point]
            path_close_rock_point.append(rock_location)
            #mark as visited
            visited[new_rock_location[0]][new_rock_location[1]] = True
            player_path.extend(path_close_rock_point)
            self.getShortestPathsFromRocks_recursive(new_rock_location, rock_location, visited, player_path, shortest_paths)
            visited[new_rock_location[0]][new_rock_location[1]] = False
            for _ in range(len(path_close_rock_point)):
                player_path.pop()
            self.moveBackRock(self.getBlock(close_to_rock_point), self.getBlock(rock_location))

            


            
         


    def getCloserRocks(self, position):
        """
        Returns a list with all the blocks near the position that had a rock and can be pushed 
        Parameters
        ----------
        position : Tuple(Int)
            The position to check for rocks
            
        Returns
        -------
        List(BlockInterface)
            A list of all the rocks that are closer to the given position
        """
        row, col = position
        points = [
            ((row + 1, col), (row + 2, col)),
            ((row - 1, col), (row - 2, col)),
            ((row, col + 1), (row, col + 2)),
            ((row, col - 1), (row, col - 2)),
            ]
        closeRocks = []
        for rock_position, opposite_point in points:
            if not self.isInBoard(rock_position):
                continue
            if not self.isInBoard(opposite_point):
                continue
            if not self.getBlock(rock_position).has_a_rock():
                continue
            if self.getBlock(rock_position).get_rock().wasVisited(opposite_point):
                continue
            if self.getBlock(rock_position).can_contain_rock() and self.getBlock(rock_position).has_a_rock():
                continue
            if not self.getBlock(opposite_point).can_contain_rock():
                if not self.getBlock(opposite_point).get_is_walkable():
                    continue
            closeRocks.append(self.getBlock(rock_position))
        return closeRocks

    def getMovablePointRock(self, position):
        """
        Given a rock position, return the point where the rock can be moved
        ----------
        position : Tuple(Int)
            The position to check for rocks
            
        Returns
        -------
        List(Tuple(Int))
            A list of all the points where the rock can be pushed 
        """
        row, col = position
        points = [
            ((row + 1, col), (row - 1, col)),
            ((row, col + 1), (row, col - 1)),
            ((row - 1, col), (row + 1, col)),
            ((row, col - 1), (row, col + 1)),
            ]
        possible_player_points = []
        for player_position, opposite_point in points:
            if not self.isInBoard(player_position):
                continue
            if not self.isInBoard(opposite_point):
                continue
            if self.getBlock(opposite_point).has_a_rock():
                continue
            if not self.getBlock(player_position).get_is_walkable():
                continue
            if not self.getBlock(opposite_point).can_contain_rock():
                if not self.getBlock(opposite_point).get_is_walkable():
                    continue
            possible_player_points.append(player_position)
        return possible_player_points
        


    def generateBoard(self, gameMap, gameBoard):
        """
        Generates the board based on the given map
        """
        self.board = []
        generator = BlockGenerator(gameBoard)
        #generate an empty matrix with the same size as the map
        for row in range(len(gameMap)):
            self.board.append([])
            for col in range(len(gameMap[row])):
                self.board[row].append(None)

        #generate the board
        for row in range(len(gameMap)):
            for col in range(len(gameMap[row])):
                self.board[row][col] = generator.generate(gameMap[row][col])
                #update player location
                if gameMap[row][col] == 'P':
                    self.characterLocation = (row, col)
                #Update arrival location
                if gameMap[row][col] == 'A':
                    self.destinationLocation = (row, col)
                #update diamonds
                if gameMap[row][col] == 'D':
                    self.remainingDiamonds += 1
                #update visited blocks if is a rock
                if gameMap[row][col] == 'R':
                    self.board[row][col].get_rock().visit((row, col))
                #add the locaation 
                self.board[row][col].set_location((row, col))
        return self.board   


    def updateDestinationState(self):
        destination_row, destination_col = self.destinationLocation
        destination_block = self.board[destination_row][destination_col]

        if self.remainingDiamonds == 0:
            destination_block.set_is_walkable(True)
        else:
            destination_block.set_is_walkable(False)


    def generateVisitedMatrix(self):
        """
        Generates a matrix of the same size as the board, but filled with False
        """
        return [[False for x in range(len(self.board[0]))] for y in range(len(self.board))]

        

    def isSolved(self):
        """
        Returns True if the map is solved
        """
        if self.characterLocation != self.destinationLocation:
            return False
        print("is solved")
        return True
            
    def getInstructionsLenght(self, instructions):
        """
        Returns the lenght of the instructions
        """
        lenght = 0
        for instruction in instructions:
            lenght += len(instruction)
        return lenght

    
    def getSpecialBlocksPath(self):
        """
        starting from the character location, get all the speacial blocks for example diamonds, keys, etc
        """
        #if the character is alrady at the destination, return an empty list
        if self.characterLocation == self.destinationLocation:
            return []


        #list of reachable blocks each element is dict with keys: block, location where location is a list of charters, each character is a location in the board
        reachableBlocks = {} 
        #matrix to keep track of visited blocks
        visited = self.generateVisitedMatrix()
        visited[self.characterLocation[0]][self.characterLocation[1]] = True
        #seacht the shortest path to each reachable location
        self.getSpecialBlocksRecursive(self.characterLocation, visited, reachableBlocks, [self.characterLocation])

        #convert the reachable blocks to a list of ReachableBlock
        reachableBlocksList = []
        for blocks, path in reachableBlocks.items():
            block_from, block_to = blocks
            reahableBlock = ReachableBlock(block_from, block_to, path)
            reachableBlocksList.append(reahableBlock)
        
        #sort the list based on the lenght of the path
        reachableBlocksList.sort(key=lambda x: x.distance)
        return reachableBlocksList
    
    
    
    def getSpecialBlocksRecursive(self, location, visited, reachableBlocks, path):
        """
        Recursive function to get all the special blocks that are reachable from the given location

        Parameters
        ----------
        location : Tuple(Int)
            The location to start from
        visited : List(List(Bool))
            A matrix of the same size as the board, but filled with False
        reachableBlocks : Dict(BlockIterface, List(Tuple(Int)))
            A list of all the reachable blocks
        path : List(Tuple(Int))
            The path to the current location
        
        Returns
        -------
        Dict(BlockIterface, List(Tuple(Int)))
        """

        #get the current block
        block = self.getBlock(location)
        curr_row, curr_col = location

        def _updateReachableBlocks(blockFrom, BlockTo):
            """
            Updates the reachable blocks list
            """
            if (blockFrom, BlockTo) not in reachableBlocks:
                    reachableBlocks[(blockFrom, BlockTo)] = copy.deepcopy(path)
            elif len(path) < len(reachableBlocks[(blockFrom, BlockTo)]):
                        reachableBlocks[(blockFrom, BlockTo)] = copy.deepcopy(path)

        #if the current block is a special block, add it to the reachable blocks
        if block.isReachable():
            #save a tuple with the first postion current block and the second position next block
            if block.get_is_collectable():
                _updateReachableBlocks(block, block)
                return

            close_rocks = block.get_close_rocks()

            for rock in close_rocks:
                _updateReachableBlocks(block, rock)
            
            
            
        #get the next blocks
        next_blocks = self.getNeighbors(location, visited)

        #for each next block, call the function recursively
        for next_block in next_blocks:
            path.append(next_block)
            visited[curr_row][curr_col] = True
            self.getSpecialBlocksRecursive(next_block, visited, reachableBlocks, path)
            path.pop()
            visited[curr_row][curr_col] = False

        
    def getNeighbors(self, location, visited):
        """
        returns a list of neighbors of the given location, the neighbords are tuples of (row, col) of blocks that are walkable
        Parameters
        ----------
        location : Tuple(Int)
            The location to get the neighbors from
        visited : List(List(Bool))
            A matrix of the same size as the board, but filled with False
        Returns
        -------
        List(Tuple(Int))
        """
        neighbors = []
        curr_row, curr_col = location
        for next_row, next_col in [[curr_row + 1, curr_col], [curr_row - 1, curr_col], [curr_row, curr_col + 1], [curr_row, curr_col - 1]]:
            if not(next_row >= 0 and next_row < len(self.board) and next_col >= 0 and next_col < len(self.board[0])):
                continue
            if visited[next_row][next_col]:
                continue
            neighbors.append((next_row, next_col))

        #get the block of each neighbor and check if it is walkable
        neighbors = [neighbor for neighbor in neighbors if self.board[neighbor[0]][neighbor[1]].get_is_walkable()]

        return neighbors
    
    def __str__(self) -> str:
        map_str = ""
        for row in self.board:
            for block in row:
                if block.has_a_rock():
                    map_str += "R"
                elif self.characterLocation == block.get_position():
                    map_str += "P"
                elif isinstance(block, Diamond):
                    if block.get_is_collectable():
                        map_str += "D"
                    else:
                        map_str += "F"
                else:
                    map_str += str(block)
                map_str += " "
            map_str += "\n"
        return map_str

    def get_new_rock_location(self, character_location_block, rock_location_block):
        """
        Returns the new location of the rock after moving it in the given direction
        Parameters
        ----------
        character_location_block : BlockInterface
            The block that the character is on
        rock_location_block : BlockInterface
            The block that the rock is on
        """
        
        direction = getDirectionOfMovement(character_location_block.get_position(), rock_location_block.get_position())
        row, col = rock_location_block.get_position()
        if direction == "L":
            col -= 1
        elif direction == "R":
            col += 1
        elif direction == "U":
            row -= 1
        elif direction == "D":
            row += 1
        return self.getBlock((row, col))

    def moverRock(self, character_location_block, rock_location_block):
        """
        Taking the character location and the rock location, move the rock to oposite direction of the character
        Parameters
        ----------
        character_location_block : BlockInterface
            The block where the character is located
        rock_location_block : BlockInterface
            The block where the rock is located
        """
        #get the new rock position after the movement
        new_rock_location_block = self.get_new_rock_location(character_location_block, rock_location_block)

        #move the rock
        rock = rock_location_block.get_rock() #take the rock from the rock location
        new_rock_location_block.set_rock(rock) #add the rock to the new rock location
        rock.visit(new_rock_location_block.get_position()) #update the visited matrix from the rock 
        rock_location_block.set_rock(None) #remove the rock form the rock location
        return new_rock_location_block
 
    def moveBackRock(self, character_location_block, rock_location_block):
        """
        Taking the character location and the rock location, move the rock to oposite direction of the character
        Parameters
        ----------
        character_location_block : BlockInterface
            The block where the character is located
        rock_location_block : BlockInterface
            The block where the rock is located
        """
        #get the new rock position after the movement
        new_rock_location_block = self.get_new_rock_location(character_location_block, rock_location_block)
        #move the rock
        rock = new_rock_location_block.get_rock() #take the rock from the rock location
        new_rock_location_block.set_rock(None) #remove the rock form the rock location
        rock_location_block.set_rock(rock) #add the rock to the new rock location
        rock.unVisit(new_rock_location_block.get_position()) #update the visited matrix from the rock 


    def moveBack(self, reachableBlock, step):
        """
        Moves the character back to the given block reverting the changes made to the board
        """
        if reachableBlock.block_from == reachableBlock.block_to:
            #interact with the las element of the path
            reachableBlock.block_from.unInteract(step)
            
            #change the character location
            path = reachableBlock.path
            self.characterLocation = path[0]

            #unwalk over the path between the character and the block
            for i in range(1, len(path) - 1):
                row_point = path[i][0]
                col_point = path[i][1]
                self.board[row_point][col_point].unwalkOver()
        else:
            #unInteract wiht the last element of the path
            reachableBlock.block_to.unInteract(step)
            #unMove the rock 
            self.moveBackRock(reachableBlock.block_from, reachableBlock.block_to)
            path = reachableBlock.path
            self.characterLocation = path[0]


    def moveTo(self, reachableBlock, step):
        """
        Moves the character to the given block

        Parameters:
        ----------
        reachableBlock : ReachableBlock
            the block to move to, its compossed by an array of points of the map where the character moves
            for example: [(3, 3), (2, 3), (1, 3), (0, 3)] it means that the character moves from (3, 3) to (0, 3)
            incluiding the block (0, 3) in the path
        """
        if reachableBlock.block_from == reachableBlock.block_to:
            #interact wiht the last element of the path
            reachableBlock.block_from.interact(step)
            #move the character
            self.characterLocation = reachableBlock.block_from.get_position()
            #move the blocks
            path = reachableBlock.path
            for i in range(1, len(path) - 1):
                row_point = path[i][0]
                col_point = path[i][1]
                self.board[row_point][col_point].walkOver()
            #get the list of moves 
            return  reachableBlock.list_movements
             
        else:
            #move the rock 
            self.moverRock(reachableBlock.block_from, reachableBlock.block_to)
            #interact wiht the last element of the path
            reachableBlock.block_to.interact(step)
            #move the character
            self.characterLocation = reachableBlock.block_to.get_position()

            #move the blocks
            path = reachableBlock.path
            for i in range(1, len(path)):
                row_point = path[i][0]
                col_point = path[i][1]
                self.board[row_point][col_point].walkOver()
            #get the list of moves 
            list_movements =  reachableBlock.list_movements
            #add the movement result of moving the rock
            list_movements.append(getDirectionOfMovement(reachableBlock.block_from.get_position(), reachableBlock.block_to.get_position()))
            return list_movements



        
        

    def getShortestSolutionPath(self):
        """
        Returns the shortest path to the solution
        This path should collect all the diamonds 
        """
        instructions = [] 
        visited = {}
        step = 1
        self.getSinlgeSolutionPathRecursive(instructions, visited, step)
        #self.getBestSolutionPathRecursive(instructions, best_instructions)
        return instructions

    def getSinlgeSolutionPathRecursive(self, instructions, visited, step):
        """
        Recursive function to get the shortest path to the solution
        """
        #if the character is alrady at the destination, return an empty list
        if self.isSolved():
            return True
        #get the reachable blocks
        reachableBlocks = self.getSpecialBlocksPath()


        #for each reachable block, move to it and call the function recursively
        for i, ReachableBlock in enumerate(reachableBlocks):
            list_movements = self.moveTo(ReachableBlock, step)
            instructions.extend(list_movements)
            #call the function recursively
            if self.getSinlgeSolutionPathRecursive(instructions, visited, step + 1):
                return True
            #move back
            self.moveBack(ReachableBlock, step)
            #remove the moves from the instructions
            for _ in range(len(list_movements)):
                instructions.pop()

        return False

    def getShortestPathFromTo(self, from_location, to_locations):
        """
        Returns the shortest path from the given location to the given location
        Parameters
        ----------
        from_location : tuple
            The location from where the path starts
        to_locations : list
            The list of locations where the path ends
        """
        visited = self.generateVisitedMatrix() 
        visited[from_location[0]][from_location[1]] = 1
        path = [from_location]
        shortest_path = {}
        self.getShortestPathFromToRecursive(from_location, to_locations, visited, path, shortest_path)
        return shortest_path


    def getShortestPathFromToRecursive(self, from_location, to_locations, visited, path, shortest_path):
        """
        Recursive function to get the shortest path from the given location to the given location
        Parameters
        ----------
        from_location : tuple
            The location from where the path starts
        to_locations : list
            The list of locations where the path ends
        visited : list
            The matrix of visited blocks
        path : list
            The path from the start to the current block
        shortest_path : list
            The shortest path from the start to the current block
        """
        #if the character is alrady at the destination, return an empty list
        if from_location in to_locations:
            if from_location not in shortest_path:
                shortest_path[from_location] = copy.deepcopy(path)
            else:
                if len(path) < len(shortest_path[from_location]):
                    shortest_path[from_location] = copy.deepcopy(path)
             
        
        #get neighbors
        neighbors = self.getNeighbors(from_location, visited)
        #for each neighbor, move to it and call the function recursively
        for row, col  in neighbors:
            if not visited[row][col]:
                visited[row][col] = True
                path.append((row, col))
                self.getShortestPathFromToRecursive((row, col), to_locations, visited, path, shortest_path)
                path.pop()
                visited[row][col] = False
    
    def getSpecialBlocksPathsWithRock(self, rock):
        """
        Returns the list of reachable blocks from the current location of the character the reachable blocks are holes or buttons
        Parameters
        ----------
        rock : Rock
            The rock to move
        """
        #get the reachable blocks
        reachableBlocks = self.getSpecialBlocksPath()
        #filter the reachable blocks
        reachableBlocks = list(filter(lambda x: x.block_from == rock or x.block_to == rock, reachableBlocks))
        return reachableBlocks

    def isInBoard(self, point):
        """
        Checks if the given point is in the board
        """
        rows = len(self.board)
        cols = len(self.board[0])
        return point[0] >= 0 and point[0] < rows and point[1] >= 0 and point[1] < cols
        
                
    



In [168]:
#test for get shortest path from rocks 
l = [['C', 'C', 'C', 'C', 'C', 'C'],
     ['C', 'C', 'C', 'C', 'C', 'C'],
     ['C', 'C', 'R', 'C', 'C', 'H'],
     ['C', 'C', 'R', 'C', 'H', 'C'],]

board = GameBoard(l)
board.getShortestPathsFromRocks((2, 2), (0,0))

{(2, 5): [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4)],
 (3, 4): [(0, 0),
  (1, 0),
  (2, 0),
  (2, 1),
  (2, 2),
  (2, 3),
  (1, 3),
  (1, 4),
  (2, 4)]}

In [125]:
#test for getMovesToReachBlock
l = [['C', 'C', 'R', 'C', 'C', 'C'],
     ['C', 'C', 'C', 'C', 'C', 'C'],
     ['C', 'C', 'R', 'C', 'C', 'C'],
     ['C', 'C', 'R', 'C', 'C', 'C'],]

board = GameBoard(l)
movable_points_rock = board.getMovablePointRock((2, 2))
shortest_paths = board.getShortestPathFromTo((0, 0), movable_points_rock)
#iterate over the shortest paths
for point in shortest_paths:
     path = shortest_paths[point]
     print(path, point)

[(0, 0), (1, 0), (2, 0), (2, 1)] (2, 1)
[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (2, 3)] (2, 3)


In [88]:
#test for getShortestPathFromTo
l = [['C', 'C', 'R', 'C', 'C', 'C'],
     ['C', 'P', 'R', 'C', 'C', 'C'],
     ['C', 'C', 'R', 'C', 'C', 'C'],
     ['C', 'C', 'R', 'C', 'C', 'C'],]

board = GameBoard(l)
board.getShortestPathFromTo((1, 1), [ (0, 2), (2, 3)])

{}

In [216]:
#test for moving rock and collection 
l = [['M', 'M', 'M', 'M', 'M', 'M'],
    ['M', 'P', 'R', 'C', 'D', 'C'],
    ['M', 'M', 'M', 'M', 'M', 'M'],]

board = GameBoard(l)
board.moveTo(board.getSpecialBlocksPath()[0])
print(board.remainingDiamonds)
board.moveTo(board.getSpecialBlocksPath()[0])
print(board.remainingDiamonds)
board.moveTo(board.getSpecialBlocksPath()[0])
print(board.remainingDiamonds)
print(board)

1
1
0
W W W W W W 
W F F F P R 
W W W W W W 



In [76]:
#test for level 4
Level4 = [
          ['M', 'H', 'C', 'C', 'C', 'M', 'C', 'C', 'R', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'P', 'M', 'M', 'M'],
          ['M', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'M'],
          ['M', 'C', 'R', 'C', 'C', 'C', 'C', 'C', 'C', 'M'],
          ['M', 'M', 'C', 'C', 'R', 'M', 'H', 'M', 'C', 'M'],
          ['M', 'M', 'M', 'H', 'D', 'M', 'A', 'M', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'D', 'M', 'M', 'M', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M']]

Level5 = [['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'A', 'M', 'M', 'M', 'M', 'M', 'D', 'D', 'M'],
          ['M', 'P', 'S', 'C', 'C', 'L', 'M', 'D', 'D', 'M'],
          ['M', 'S', 'M', 'M', 'R', 'M', 'M', 'M', 'K', 'M'],
          ['M', 'D', 'M', 'C', 'C', 'C', 'M', 'M', 'H', 'M'],
          ['M', 'D', 'M', 'C', 'C', 'C', 'C', 'C', 'C', 'M'],
          ['M', 'H', 'M', 'C', 'C', 'C', 'C', 'C', 'C', 'M'],
          ['M', 'C', 'C', 'C', 'C', 'C', 'M', 'C', 'C', 'M'],
          ['M', 'C', 'C', 'C', 'C', 'M', 'M', 'M', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'C', 'M', 'M', 'M', 'R', 'M'],
          ['M', 'M', 'M', 'M', 'C', 'S', 'S', 'C', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'D', 'D', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M']]

board = GameBoard(matrix.Level4)
m = board.getSpecialBlocksPath()
print(board.getShortestSolutionPath())

is solved
['R', 'R', 'R', 'R', 'D', 'D', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'L', 'D', 'D', 'L', 'L', 'L', 'L', 'L', 'D', 'R', 'U', 'R', 'D', 'D', 'D', 'R', 'U', 'D', 'D', 'U', 'U', 'L', 'U', 'R', 'R', 'U', 'R', 'D', 'D', 'D']


In [169]:
#test for level 3
import matrix

Level1 = [['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'C', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'C', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'P', 'D', 'D', 'D', 'C', 'C', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'D', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'D', 'C', 'M'],
          ['M', 'C', 'D', 'C', 'D', 'D', 'C', 'C', 'C', 'M'],
          ['M', 'C', 'D', 'C', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'D', 'C', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'C', 'D', 'D', 'C', 'C', 'C', 'M'],
          ['M', 'M', 'M', 'C', 'C', 'M', 'C', 'A', 'C', 'M'],
          ['M', 'C', 'C', 'C', 'C', 'M', 'C', 'C', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M']]

Level3 = [['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'C', 'C', 'M', 'C', 'M', 'C', 'D', 'M'],
          ['M', 'P', 'C', 'M', 'M', 'C', 'K', 'C', 'A', 'M'],
          ['M', 'C', 'C', 'S', 'C', 'C', 'M', 'C', 'D', 'M'],
          ['M', 'M', 'S', 'M', 'K', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'C', 'M', 'C', 'C', 'M', 'D', 'D', 'M'],
          ['M', 'C', 'D', 'M', 'C', 'C', 'K', 'L', 'D', 'M'],
          ['M', 'M', 'C', 'M', 'C', 'C', 'M', 'D', 'D', 'M'],
          ['M', 'L', 'C', 'M', 'D', 'C', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'M', 'M', 'C', 'C', 'M', 'C', 'C', 'M'],
          ['M', 'D', 'S', 'S', 'C', 'C', 'K', 'L', 'D', 'M'],
          ['M', 'C', 'M', 'M', 'M', 'L', 'M', 'C', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M']]

Level2 = [['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'C', 'D', 'C', 'C', 'M', 'W', 'W', 'M'],
          ['M', 'C', 'D', 'C', 'D', 'C', 'M', 'W', 'W', 'M'],
          ['M', 'S', 'M', 'M', 'M', 'C', 'M', 'M', 'C', 'M'],
          ['M', 'D', 'W', 'W', 'M', 'D', 'C', 'D', 'C', 'M'],
          ['M', 'D', 'W', 'W', 'M', 'D', 'C', 'D', 'C', 'M'],
          ['M', 'C', 'M', 'M', 'M', 'S', 'M', 'W', 'W', 'M'],
          ['M', 'C', 'M', 'C', 'D', 'S', 'M', 'W', 'W', 'M'],
          ['M', 'C', 'M', 'A', 'M', 'P', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'M', 'M', 'M', 'C', 'C', 'C', 'C', 'M'],
          ['M', 'D', 'D', 'S', 'S', 'C', 'C', 'C', 'C', 'M'],
          ['M', 'D', 'D', 'M', 'M', 'C', 'C', 'C', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M']]

board = GameBoard(Level4)
print(board.getShortestSolutionPath())


is solved
['D', 'L', 'L', 'L', 'L', 'L', 'D', 'R', 'U', 'R', 'D', 'D', 'D', 'R', 'U', 'D', 'D', 'U', 'U', 'L', 'U', 'R', 'R', 'U', 'R', 'D', 'D', 'D']


In [49]:
#test for reachable rocks 
level = [
    ['C', 'R', 'C', 'C', 'C'],
    ['C', 'R', 'D', 'C', 'C'],
    ['C', 'R', 'L', 'R', 'C'],
    ['C', 'R', 'H', 'C', 'C'],
    ['P', 'R', 'C', 'C', 'C'],
]

board = GameBoard(level)

print(board.getBlock((0,1)).isReachable() == False)
print(board.getBlock((1,1)).isReachable() == False)
print(board.getBlock((2,1)).isReachable() == False)
print(board.getBlock((3,1)).isReachable() == False)
print(board.getBlock((4,1)).isReachable() == False)
print(board.getBlock((0,0)).isReachable() == True)
print(board.getBlock((1,0)).isReachable() == True)
print(board.getBlock((2,0)).isReachable() == True)
print(board.getBlock((3,0)).isReachable() == False)
print(board.getBlock((4,0)).isReachable() == True) # the player is not a reachable point


True
True
True
True
True
True
True
True
True
True


In [42]:
#test for moving rocks 
level = [
    ['C', 'R', 'C', 'C', 'C'],
    ['C', 'R', 'D', 'C', 'C'],
    ['C', 'R', 'L', 'R', 'C'],
    ['P', 'R', 'H', 'C', 'C'],
]

board = GameBoard(level)

print("Test for unblocked movement")
block_from =  board.getBlock((0, 0))
block_to = board.getBlock((0, 1))
new_rock = board.getBlock((0,2))
board.moverRock(block_from, block_to)
print(block_to.has_a_rock() == False)
print(block_to.isReachable() == True)
print(block_to.get_is_walkable() == True)
print(new_rock.get_is_walkable() == False)
print(new_rock.has_a_rock() == True)
print(new_rock.isReachable() == False)


print("Test for key movement")
block_from =  board.getBlock((2, 0))
block_to = board.getBlock((2, 1))
new_rock = board.getBlock((2,2))
board.moverRock(block_from, block_to)
print(block_to.has_a_rock() == False)
print(",", block_to.isReachable() == True)
print(new_rock.has_a_rock() == True)
print(new_rock.isReachable() == False)
print(new_rock.get_is_walkable() == False)
print(new_rock.has_a_rock() == True)
print(new_rock.isReachable() == False)


print("Test for dimond movement")
block_from =  board.getBlock((1, 0))
block_to = board.getBlock((1, 1))
new_rock = board.getBlock((1,2))
board.moverRock(block_from, block_to)
print(block_to.has_a_rock() == False)
print(block_to.isReachable() == True)
print(new_rock.has_a_rock() == True)
print(new_rock.isReachable() == False)
print(new_rock.get_is_walkable() == False)
print(new_rock.has_a_rock() == True)
print(new_rock.isReachable() == False)




Test for unblocked movement
True
True
True
True
True
True
Test for key movement
True
, True
True
True
True
True
True
Test for dimond movement
True
True
True
True
True
True
True


In [92]:
#get special block path recursive
level = [
    ['C', 'C', 'C', 'C', 'C'],
    ['C', 'C', 'M', 'C', 'C'],
    ['C', 'R', 'L', 'R', 'C'],
    ['C', 'M', 'C', 'C', 'C'],
    ['P', 'C', 'C', 'C', 'C'],
]

board = GameBoard(level)
a = board.getSpecialBlocksPath()[0]
a.block_from.get_position()




(2, 0)

In [30]:
# test for close rocks 
level = [
    ['C', 'C', 'C', 'C', 'C'],
    ['C', 'C', 'M', 'C', 'C'],
    ['C', 'R', 'C', 'R', 'C'],
    ['C', 'C', 'C', 'C', 'C'],
    ['P', 'C', 'C', 'C', 'C'],
]

board = GameBoard(level)

board.getCloserRocks((2,2))

[Floor at (2, 3), Floor at (2, 1)]

In [76]:
#test  for reachable blocks using rocks
level = [
    ['C', 'C', 'C', 'C', 'C'],
    ['C', 'C', 'M', 'C', 'C'],
    ['C', 'S', 'R', 'D', 'C'],
    ['C', 'C', 'C', 'C', 'C'],
    ['P', 'C', 'C', 'C', 'C'],
]

board = GameBoard(level)

rock_row = 2
rock_col = 2

#test for initializtion
print("correct initialized", isinstance(board.board[rock_row][rock_col].get_rock(), Rock))
print("the rock shoukd not be walkable", not board.board[rock_row][rock_col].get_is_walkable())

#test for has a rock 
board.updateReachableBlocksCloseToRocks(rock_row, rock_col)

left_neighbour = board.board[rock_row][rock_col - 1]
right_neighbour = board.board[rock_row][rock_col + 1]
up_neighbour = board.board[rock_row - 1][rock_col]
down_neighbour = board.board[rock_row + 1][rock_col]

print("The blocks has a rock test......")
print("correct left neighbour", isinstance(left_neighbour.get_close_rocks()[0], Rock))
print("correct right neighbour", isinstance(right_neighbour.get_close_rocks()[0], Rock))
print("correct down neighbour", down_neighbour.get_close_rocks() == [])
print("correct up neighbour", down_neighbour.get_close_rocks() == [])


board.getShortestSolutionPath()



correct initialized True
the rock shoukd not be walkable False
The blocks has a rock test......
correct left neighbour True
correct right neighbour True
correct down neighbour True
correct up neighbour True


[]

In [8]:
a = ['D', 'R', 'D', 'D', 'D', 'D', 'D', 'L', 'D', 'D', 'R', 'R', 'R', 'U', 'U', 'U', 'U', 'U', 'U', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'D', 'U', 'R', 'R', 'R', 'L', 'L', 'L', 'U', 'U', 'U', 'U', 'R', 'R', 'D', 'R', 'U', 'U', 'L', 'D', 'L', 'L', 'U', 'L', 'U', 'U', 'R', 'U', 'R', 'R', 'D', 'R', 'L', 'U', 'U', 'R', 'D']

In [144]:
#test for key doors
level = [
    ['P', 'D', 'L', 'K', 'A']
]

board = GameBoard()
board.initialize(level, board)


moves = board.getSpecialBlocksPath()
board.moveTo(moves[0])
moves2 = board.getSpecialBlocksPath()
board.moveTo(moves2[0])
moves3 = board.getSpecialBlocksPath()
board.moveTo(moves3[0])
moves4 = board.getSpecialBlocksPath()
board.moveTo(moves4[0])

board.moveBack(moves4[0])
print("has key", board.has_key == False)
board.moveBack(moves3[0])
print("has key", board.has_key == True)
print("Is walkable", board.board[0][3].is_walkable == True)
board.moveBack(moves2[0])
board.moveBack(moves[0])
print(board.remainingDiamonds == 1)



has key True
has key True
has key True
True


In [88]:
#test for go back spears FAILED, IMPROVE LATER
#test for move back
Level1 = [
          ["M","M","M","D"],
          ["A","D","S","D"],
          ["M","M","P","C"],
          ]

board = GameBoard()
board.initialize(Level1, board)
board.getSpecialBlocksPath()

[[(2, 2), (1, 2), (1, 3)], [(2, 2), (1, 2), (1, 1)]]

In [31]:
#test for move back
Level1 = [
          ["A","C","D"],
          ["C","C","D"],
          ["C","P","C"],
          ]

board = GameBoard()
board.initialize(Level1, board)

m1 = board.getSpecialBlocksPath()
board.moveTo(m1[0])
print("1 move")
print(board.remainingDiamonds)
print(board.characterLocation)
m2 = board.getSpecialBlocksPath()
board.moveTo(m2[0])
print("2 move")
print(board.remainingDiamonds)
print(board.characterLocation)
m3 = board.getSpecialBlocksPath()
board.moveTo(m3[0])
print("3 move")
print(board.remainingDiamonds)
print(board.characterLocation)
print("first move back")
board.moveBack(m3[0])
print(board.remainingDiamonds)
print(board.characterLocation)
print("second move back")
board.moveBack(m2[0])
print(board.remainingDiamonds)
print(board.characterLocation)
print("third move back")
board.moveBack(m3[0])
print(board.remainingDiamonds)
print(board.characterLocation)


1 move
1
(0, 2)
2 move
0
(1, 2)
3 move
0
(0, 0)
first move back
1
(1, 2)
second move back
2
(0, 2)
third move back
2
(1, 2)


In [90]:

# test for solution
import matrix

board = GameBoard()
board.initialize(matrix.Level3, board)
print(board.getShortestSolutionPath())
print(board.characterLocation)
print(board.isSolved())



0 11 [(4, 1), (5, 1), (5, 2), (6, 2), (7, 2), (8, 2)]
0 10 [(8, 2), (9, 2), (10, 2), (10, 1)]
0 10 [(10, 1), (11, 1), (12, 1)]
0 9 [(12, 1), (12, 2), (12, 3), (12, 4), (11, 4), (10, 4)]
[]
(4, 1)
False


In [30]:
#test for get neighbors
mapReachable = [
    ['C', 'M', 'P'],
    ['A', 'C', 'C'],
    ['C', 'C', 'C'],
]

visited = [
    [False, False, False],
    [False, False, True],
    [False, False, False],
]

board = GameBoard()
board.initialize(mapReachable, board)

board.getNeighbors([1, 1], visited)

[(2, 1)]

In [29]:
#test for solve the map
mapReachable = [
    ['A', 'S', 'D', 'P'],
]

board = GameBoard()
board.initialize(mapReachable, board)
block = board.getSpecialBlocksPath()
board.moveTo(block[0])
print(board.characterLocation)
block = board.getSpecialBlocksPath()
board.moveTo(block[0])
print(board.characterLocation)
print(board.isSolved())



(0, 2)
(0, 0)
True


In [13]:
#test for movement 
mapReachable = [
    ['L', 'C', 'C', 'D'],
    ['C', 'M', 'C', 'C'],
    ['C', 'M', 'C', 'S'],
    ['A', 'C', 'C', 'P'],
]

board = GameBoard()
board.initialize(mapReachable, board)
block = board.getSpecialBlocksPath()
print(block)
list_movements = board.moveTo(block[1])
print("test list of movements correct")
print(list_movements == ['U', 'U', 'L', 'U', 'L', 'L']) 
print("test for player location")
print(board.characterLocation == (0,0))
print(board.board[2][3].is_walkable == False)



TypeError: getNeighbors() missing 1 required positional argument: 'visited'

In [14]:
#test walk over and undwalkover
gameBoard = GameBoard()

#TEST 1 for key block
keyBlock = BlockGenerator().generate('L', gameBoard)
print("Test for key block")
print(keyBlock.isReachable() == 1)
print(keyBlock.is_collectable == 1)
keyBlock.walkOver()
print(gameBoard.has_key == True) #despues de pisado debe tener una llave
print(keyBlock.isReachable() == 0) #no puede ser alcanzacble
print(keyBlock.is_collectable == 0) #no puede ser recogido
keyBlock.unwalkOver()
print(gameBoard.has_key == False) #despues de pisado no debe tener una llave
print(keyBlock.isReachable() == 1) #puede ser alcanzacble
print(keyBlock.is_collectable == 1) #puede ser recogido

#TEST 2 for diamond block
diamondBlock = BlockGenerator().generate('D', gameBoard)
print("Test for diamond block")
gameBoard.remainingDiamonds = 5
print(diamondBlock.isReachable() == 1)
print(diamondBlock.is_collectable == 1)
diamondBlock.walkOver()
print(diamondBlock.isReachable() == 0)
print(diamondBlock.is_collectable == 0)
print(gameBoard.remainingDiamonds == 4)
diamondBlock.unwalkOver()
print(diamondBlock.isReachable() == 1)
print(diamondBlock.is_collectable == 1)
print(gameBoard.remainingDiamonds == 5)

#test 3 for spears block
spearsBlock = BlockGenerator().generate('S', gameBoard)
print("Test for spears block")
print(spearsBlock.is_walkable == True)#puede ser pisado
spearsBlock.walkOver()
print(spearsBlock.is_walkable == False)#no puede ser pisado
spearsBlock.unwalkOver()
print(spearsBlock.is_walkable == True)#puede ser pisado






Test for key block
True
True
True
True
True
True
True
True
Test for diamond block
True
True
True
True
True
True
True
True
Test for spears block
True
True
True


In [8]:
# test for generator class
gameBoard = GameBoard()
generator = BlockGenerator()
print(isinstance(generator.generate("L", gameBoard), Key) == True)
print(isinstance(generator.generate("C", gameBoard), Floor) == True)
print(isinstance(generator.generate("A", gameBoard), Destination) == True)

True
True
True


In [ ]:
# test for direction of movement function
board = GameBoard()
print( board.getDirectionOfMovement((0, 0), (0, 1)) == "R")
print( board.getDirectionOfMovement((0, 0), (0, -1)) == "L")
print( board.getDirectionOfMovement((0, 0), (1, 0)) == "D")
print( board.getDirectionOfMovement((0, 0), (-1, 0)) == "U")

In [104]:
#test for is solved function
mapReachable = [
    ['C', 'C',],
    ['A', 'P',],
]

board = GameBoard()
board.initialize(mapReachable, board)

print(board.destinationLocation == (1, 0))
print(board.characterLocation == (1, 1))
print(board.remainingDiamonds == 0)
print(board.isSolved() == False)
board.characterLocation = (1, 0)
print(board.isSolved() == True)
board.remainingDiamonds = 1
print(board.isSolved() == False)



TypeError: __init__() missing 1 required positional argument: 'gameMap'

In [103]:
#test of getting the reachable blocks
mapReachable = [
    ['L', 'C', 'C', 'D'],
    ['M', 'M', 'C', 'C'],
    ['C', 'M', 'C', 'C'],
    ['C', 'C', 'C', 'P'],
]

board = GameBoard()
board.initialize(mapReachable, board)

print(board.characterLocation == (3,3))
print(board.remainingDiamonds == 1)
print(board.getSpecialBlocksPath())

TypeError: __init__() missing 1 required positional argument: 'gameMap'

In [ ]:
board = GameBoard()
board.initialize(m.Level1, board)

print(board.getSpecialBlocksPath())

In [102]:
#test of generation a signle block
gameBoard = GameBoard()

#TEST 1 for exit block
exitBlock = BlockGenerator().generate('A', gameBoard)

#test to get if the exit is reachable when there are no diamonds
gameBoard.remainingDiamonds = 0
print("test 1: ", exitBlock.isReachable() == True)

#test to get if the exit is reachable when there are diamonds
gameBoard.remainingDiamonds = 1
print("test 2: ", exitBlock.isReachable() == False)

#-------------
#TEST 2 for Door Block
doorBlock = BlockGenerator().generate('K', gameBoard)

gameBoard.has_key = True
print("test 3: ", doorBlock.isReachable() == True)

gameBoard.has_key = False
print("test 4: ", doorBlock.isReachable() == False)






TypeError: __init__() missing 1 required positional argument: 'gameMap'